# Test loading MulVal results from XSB into Python

py-xsb can be installed with `pip install py-xsb`



In [5]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

In [8]:
!pip freeze

alabaster==0.7.12
attrs==19.3.0
awscli==1.16.35
Babel==2.7.0
backcall==0.1.0
beautifulsoup4==4.8.1
bleach==3.1.0
blis==0.4.1
botocore==1.12.25
bs4==0.0.1
catalogue==0.0.8
certifi==2019.9.11
chardet==3.0.4
Click==7.0
click-plugins==1.1.1
colorama==0.3.9
cycler==0.10.0
cymem==2.0.3
decorator==4.4.1
defusedxml==0.6.0
docutils==0.15.2
entrypoints==0.3
Flask==1.1.1
idna==2.8
imageio==2.6.1
imagesize==1.1.0
importlib-metadata==0.23
ipykernel==5.1.3
ipython==7.10.0
ipython-genutils==0.2.0
ipywidgets==7.5.1
itsdangerous==1.1.0
jedi==0.15.1
Jinja2==2.10.3
jmespath==0.9.4
joblib==0.14.0
json5==0.8.5
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==5.3.4
jupyter-console==6.0.0
jupyter-contrib-core==0.3.3
jupyter-contrib-nbextensions==0.5.1
jupyter-core==4.6.1
jupyter-highlight-selected-word==0.2.0
jupyter-latex-envs==1.4.6
jupyter-nbextensions-configurator==0.4.1
jupyterlab==1.2.3
jupyterlab-server==1.0.6
jupyterlab-templates==0.2.1
kiwisolver==1.1.0
lxml==4.4.2
MarkupSafe==1.1.1
matplotlib==3.1.

In [3]:

import os
import platform
import sys

import jupyter_core

sys.path.append('..')

# clone xsb from http://xsb.sourceforge.net/
# build xsb with `./configure & ./makexsb * ./makexsb dynmodule`

# pip install py-xsb of build from https://github.com/gooofy/py-xsb
from pyxsb import pyxsb_start_session, pyxsb_end_session, pyxsb_command, \
                  pyxsb_query, XSBFunctor, XSBVariable, xsb_to_json, json_to_xsb


## Set global vars here

In [4]:

BASE_DIR = '/opt/projects/diss/jupyter_nbs/mine/'
WORKING_DIR = 'data/test_001'


# to determine the location of the XSB arch dir on your machine, run this query in XSB:
# xsb_configuration:xsb_configuration(config_dir,Dir).
# XSB_ARCH_DIR = None
XSB_ARCH_DIR = '/opt/apps/xsb/XSB/config/x86_64-unknown-linux-gnu'
#print(jupyter_core.paths.jupyter_runtime_dir())
#print(JUPYTER_RUNTIME_DIR)
#jupyter_path()
#jupyter_core.paths.jupyter_path()
jupyter_core.paths.jupyter_runtime_dir()

'/home/toor/.local/share/jupyter/runtime'

## Set CWD here

In [5]:
os.chdir(BASE_DIR + WORKING_DIR)
%pwd

'/opt/projects/diss/jupyter_nbs/mine/data/test_001'

## Test XSB bridge

In [7]:

ft_txt = """
% prolog

child(anne,bridget).
child(bridget,caroline).
child(caroline,donna).
child(donna,emily).

descend(X,Y) :- child(X,Y).
descend(X,Y) :- child(X,Z), descend(Z,Y).

label('günter', "Günter").
label(anne, "Anne").
label(bridget, "Bridget").
label(caroline, "Caroline").
label(donna, "Donna").
"""

### demo using the high level XSB interface

In [11]:

# demo using the high level XSB interface
pyxsb_start_session(XSB_ARCH_DIR)

file_name = 'ft.P'
'''
w  write mode
r  read mode
a  append mode

w+  create file if it doesn't exist and open it in (over)write mode
    [it overwrites the file if it already exists]
r+  open an existing file in read+write mode
a+  create file if it doesn't exist and open it in append mode
'''
f = open(file_name, 'w+')  # open file in append mode
f.write(ft_txt)
f.close()

pyxsb_command('consult(ft).')
print("=================")
for row in pyxsb_query('label(X, L).'):
    print(u"label of %s is %s" % (row[0], row[1]))
print("=================")
# structured XSB* interface
for row in pyxsb_query(XSBFunctor('descend', [XSBVariable('X'), XSBVariable('Y')])):
    print(u"decendant of %s is %s" % (row[0], row[1]))
print("=================")
for row in pyxsb_query(u'A = 1, B = 0.5, C = "hello", D = yes, E = foo(bar), F = [1.1,2.2], G = \'günter\'.'):

    for i, r in enumerate(row):
        print(u"#%d: %-10s (type: %-20s, class: %-20s)" % (i, r, type(r), r.__class__))

    js = xsb_to_json(row)
    print("json: %s" % js)

    row2 = json_to_xsb(js)
print("=================")
pyxsb_query('catch(abort,Exception,true).')
try:
    # throw has no effect
    pyxsb_query("writeln('Trying to throw up:'), throw('test throw').")
except:
    print('Test throw -- caught')
print("=================")
# testing abort
try:
    pyxsb_query("abort('Test abort').")
except:
    print('Test abort -- caught')
print("=================")
print('Testing complex explicit module names')
pyxsb_command('assert(ppp(simplemod:abcde(p,q,r))).')
pyxsb_command('assert(ppp(mod(arg1,f(arg2)):abc(p,q,r))).')
pyxsb_command('assert(ppp(mod2(arg,g(arg)):cde(p,q,r))).')
for row in pyxsb_query("ppp(X)."):
    print('answer: ', row[0])
print("=================")
pyxsb_end_session()

label of 'günter' is "Günter"
label of 'anne' is "Anne"
label of 'bridget' is "Bridget"
label of 'caroline' is "Caroline"
label of 'donna' is "Donna"
decendant of 'anne' is 'bridget'
decendant of 'bridget' is 'caroline'
decendant of 'caroline' is 'donna'
decendant of 'donna' is 'emily'
decendant of 'anne' is 'caroline'
decendant of 'anne' is 'donna'
decendant of 'anne' is 'emily'
decendant of 'bridget' is 'donna'
decendant of 'bridget' is 'emily'
decendant of 'caroline' is 'emily'
#0: 1          (type: <class 'int'>       , class: <class 'int'>       )
#1: 0.5        (type: <class 'float'>     , class: <class 'float'>     )
#2: "hello"    (type: <class 'pyxsb.XSBString'>, class: <class 'pyxsb.XSBString'>)
#3: 'yes'      (type: <class 'pyxsb.XSBAtom'>, class: <class 'pyxsb.XSBAtom'>)
#4: foo('bar') (type: <class 'pyxsb.XSBFunctor'>, class: <class 'pyxsb.XSBFunctor'>)
#5: [1.1, 2.2] (type: <class 'list'>      , class: <class 'list'>      )
#6: 'günter'   (type: <class 'pyxsb.XSBAtom'>, c

### demo using the low level XSB interface

In [19]:
ctest_txt = """
%prolog
% this is just to test that C calling XSB works with the preprocessor
:- compiler_options([xpp_on]).
#include "char_defs.h"

p(100,aa,bb).
p(300,first,second).
p(400,wrong,one).
p(300,third,fourth).
p(X,Y,Z) :- repeat, 
	writeln('Enter a term of the form p(300,atom1,atom2). (or end_of_file.)'),
	write(': '),
	read(T), 
	(T=end_of_file -> 
		!,fail
	;
		true
	),
	T=p(X,Y,Z).
"""

In [21]:
# demo using the low level XSB interface 

file_name = 'ctest.P'
'''
w  write mode
r  read mode
a  append mode

w+  create file if it doesn't exist and open it in (over)write mode
    [it overwrites the file if it already exists]
r+  open an existing file in read+write mode
a+  create file if it doesn't exist and open it in append mode
'''
f = open(file_name, 'w+')  # open file in append mode
f.write(ctest_txt)
f.close()

pyxsb_start_session(XSB_ARCH_DIR)

# should be after start_xsb_session so that imported vars will be updated
from pyxsb import *

c2p_functor(b"consult", 1, reg_term(1))
c2p_string(b"ctest",p2p_arg(reg_term(1),1))
if xsb_command():
    raise Exception ("Error consulting ctest")

if xsb_command_string(b"consult(basics)."):
    raise Exception ("Error (string) consulting basics.")

# Create the query p(300,X,Y) and send it.
c2p_functor(b"p",3,reg_term(1))
c2p_int(300,p2p_arg(reg_term(1),1))

rcode = xsb_query()

# Print out answer and retrieve next one.
while not rcode:

    if not is_string(p2p_arg(reg_term(2),1)) and is_string(p2p_arg(reg_term(2),2)):
        print ("2nd and 3rd subfields must be atoms")
    else:
        print ("Answer: %d, %s(%s), %s(%s)" % ( p2c_int(p2p_arg(reg_term(1),1)),
                                               p2c_string(p2p_arg(reg_term(1),2)),
                                               xsb_var_string(1),
                                               p2c_string(p2p_arg(reg_term(1),3)),
                                               xsb_var_string(2)))
    rcode = xsb_next()

# Create the string query p(300,X,Y) and send it, use higher-level routines.

xsb_make_vars(3)
xsb_set_var_int(300,1)
rcode = xsb_query_string(b"p(X,Y,Z).")

# Print out answer and retrieve next one.
while not rcode:
    if not is_string(p2p_arg(reg_term(2),2)) and is_string(p2p_arg(reg_term(2),3)):
        print ("2nd and 3rd subfields must be atoms")
    else:
        print ("Answer: %d, %s, %s" % (xsb_var_int(1),
                                      xsb_var_string(2),
                                      xsb_var_string(3)))
    rcode = xsb_next()



pyxsb_query('catch(abort,Exception,true).')

pyxsb_command('consult(ft).')


pyxsb_end_session()

Answer: 300, b'first'(first), b'second'(second)
Answer: 300, b'third'(third), b'fourth'(fourth)
Answer: 300, first, second
Answer: 300, third, fourth


In [20]:
# pyxsb_query('catch(abort,Exception,true).')

In [ ]:
pyxsb_start_session(XSB_ARCH_DIR)


pyxsb_end_session()

In [8]:
pyxsb_start_session(XSB_ARCH_DIR)
pyxsb_command('consult(run).')
#pyxsb_end_session()

In [36]:
for row in pyxsb_query('hacl(A, B, C, D).'):
    print(u"hacl %s %s %s %s" % (row[0], row[1], row[2], row[3]))
    
pyxsb_end_session()




hacl '10.0.0.1/32' '10.0.0.2/32' }ކ }ކ
hacl '10.0.0.2/32' '10.0.0.3/32' }ކ }ކ
hacl '10.0.0.3/32' '10.0.0.4/32' }ކ }ކ
hacl '10.0.0.4/32' '10.0.0.5/32' 'tcp' 21


In [1]:
%pwd

'/opt/projects/diss/jupyter_nbs/mine'